In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords

In [2]:
url = 'https://finance.naver.com/news/mainnews.nhn'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
headline = []
for index, link in enumerate(soup.findAll("a", href=re.compile("^/news/news_read.nhn."))):
    if 'href' in link.attrs and link.get('title') != None:
        headline.append(link.get('title'))

In [4]:
headline

['[코스닥 메모] (11일) 변경상장=DB금융스팩6호→네온테크(상호변경) 등',
 '[유가증권 기업공시] (10일) 남광토건 등',
 '[코스닥 기업공시] (10일) 서호전기 등',
 'EY한영 성장이끈 서진석, 임기 남기고 돌연 사임 왜',
 "中공장 재개 소식에 테슬라 주가, 장외시장서 '급상승'",
 '무디스, LG화학 장기 신용등급 A3서 Baa1로↓',
 '무디스, LG화학 장기 신용등급 A3서 Baa1로↓',
 '[코스닥 공시] 국순당 / 녹십자엠에스',
 '[유가증권시장 공시] 선도전기 / 잇츠한불 / 한온시스템 등',
 '은행 방문객 40% \'뚝\'…"펜도 손세정제로 닦으며 서명"',
 '코스닥 결산 시즌, 21개사 관리종목·상장폐지 주의보',
 '테슬라 덕분에 주가 올랐다, LG화학·삼성SDI 그리고 또…',
 '[Asia마감]유동성 공급에 中증시만 ↑',
 "제약·바이오 '낚시 공시' 못한다",
 "[Asia오전]지속되는 '코로나' 공포",
 "中공장 재개 소식에 테슬라 주가, 장외시장서 '급상승'",
 '[마켓뷰] 높아지는 경기둔화 우려감… 이틀 연속 한때 2200 깨져',
 '[코스닥 공시] 국순당 / 녹십자엠에스']

In [5]:
# removing functuation
def remove_punctuation(text):
    import string
    # replacing the puntuations with no space,
    # which in effect deletes the punctuation marks
    translator = str.maketrans(' ', ' ', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

In [9]:
# extract the stop words
# extracting the stopwords
sw = stopwords.words()
print(f'Number of stopwords: {len(sw)}')
stopwords_ko = [line.replace('\n', '') for line in open('./meta/stopwords_ko', 'r', encoding='UTF8').readlines()]
print(f'Number of stopwords_ko: {len(stopwords_ko)}')
stopwords_total = stopwords_ko + sw
print(f'Number of stopwords_total: {len(stopwords_total)}')
# removing stopwords
def remove_stopwords(text):
    # removing the stop words and lowercasing the selected words
    text = [word for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

Number of stopwords: 6800
Number of stopwords_ko: 675
Number of stopwords_total: 7475


In [10]:
evaluation = pd.DataFrame()
evaluation['headline'] = headline
evaluation['tokenization'] = [re.sub(r'[^A-Za-z0-9가-힣一-龥]', ' ', i) for i in evaluation.headline]
evaluation['tokenization'] = evaluation['tokenization'].apply(remove_punctuation).apply(remove_stopwords)
evaluation['tokenization'] = [i.split(' ') for i in evaluation['tokenization']]

In [11]:
evaluation

,headline,tokenization
0,[코스닥 메모] (11일) 변경상장=DB금융스팩6호→네온테크(상호변경) 등,"[코스닥, 메모, 11일, 변경상장, DB금융스팩6호, 네온테크, 상호변경, 등]"
1,[유가증권 기업공시] (10일) 남광토건 등,"[유가증권, 기업공시, 10일, 남광토건, 등]"
2,[코스닥 기업공시] (10일) 서호전기 등,"[코스닥, 기업공시, 10일, 서호전기, 등]"
3,"EY한영 성장이끈 서진석, 임기 남기고 돌연 사임 왜","[EY한영, 성장이끈, 서진석, 임기, 남기고, 돌연, 사임, 왜]"
4,"中공장 재개 소식에 테슬라 주가, 장외시장서 '급상승'","[中공장, 재개, 소식에, 테슬라, 주가, 장외시장서, 급상승]"
5,"무디스, LG화학 장기 신용등급 A3서 Baa1로↓","[무디스, LG화학, 장기, 신용등급, A3서, Baa1로]"
6,"무디스, LG화학 장기 신용등급 A3서 Baa1로↓","[무디스, LG화학, 장기, 신용등급, A3서, Baa1로]"
7,[코스닥 공시] 국순당 / 녹십자엠에스,"[코스닥, 공시, 국순당, 녹십자엠에스]"
8,[유가증권시장 공시] 선도전기 / 잇츠한불 / 한온시스템 등,"[유가증권시장, 공시, 선도전기, 잇츠한불, 한온시스템, 등]"
9,"은행 방문객 40% '뚝'…""펜도 손세정제로 닦으며 서명""","[은행, 방문객, 40, 뚝, 펜도, 손세정제로, 닦으며, 서명]"


In [15]:
negative = [line.replace('\n', '') for line in open('./meta/negative', 'r', encoding='UTF8').readlines()]
positive = [line.replace('\n', '') for line in open('./meta/positive', 'r', encoding='UTF8').readlines()]
sentimental = pd.read_csv('./meta/sentimental', sep='\t')

In [16]:
negative[:5], positive[:5]

(['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음'],
 ['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)'])

In [17]:
sentimental.head(5)

,keyword,score
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1


In [20]:
evaluation.head()

,headline,tokenization
0,[코스닥 메모] (11일) 변경상장=DB금융스팩6호→네온테크(상호변경) 등,"[코스닥, 메모, 11일, 변경상장, DB금융스팩6호, 네온테크, 상호변경, 등]"
1,[유가증권 기업공시] (10일) 남광토건 등,"[유가증권, 기업공시, 10일, 남광토건, 등]"
2,[코스닥 기업공시] (10일) 서호전기 등,"[코스닥, 기업공시, 10일, 서호전기, 등]"
3,"EY한영 성장이끈 서진석, 임기 남기고 돌연 사임 왜","[EY한영, 성장이끈, 서진석, 임기, 남기고, 돌연, 사임, 왜]"
4,"中공장 재개 소식에 테슬라 주가, 장외시장서 '급상승'","[中공장, 재개, 소식에, 테슬라, 주가, 장외시장서, 급상승]"


In [25]:
evaluation['positive_score'] = 0
evaluation['negative_score'] = 0

In [29]:
evaluation

,headline,tokenization,positive_score,negative_score
0,[코스닥 메모] (11일) 변경상장=DB금융스팩6호→네온테크(상호변경) 등,"[코스닥, 메모, 11일, 변경상장, DB금융스팩6호, 네온테크, 상호변경, 등]",0,0
1,[유가증권 기업공시] (10일) 남광토건 등,"[유가증권, 기업공시, 10일, 남광토건, 등]",0,0
2,[코스닥 기업공시] (10일) 서호전기 등,"[코스닥, 기업공시, 10일, 서호전기, 등]",0,0
3,"EY한영 성장이끈 서진석, 임기 남기고 돌연 사임 왜","[EY한영, 성장이끈, 서진석, 임기, 남기고, 돌연, 사임, 왜]",0,0
4,"中공장 재개 소식에 테슬라 주가, 장외시장서 '급상승'","[中공장, 재개, 소식에, 테슬라, 주가, 장외시장서, 급상승]",0,0
5,"무디스, LG화학 장기 신용등급 A3서 Baa1로↓","[무디스, LG화학, 장기, 신용등급, A3서, Baa1로]",0,0
6,"무디스, LG화학 장기 신용등급 A3서 Baa1로↓","[무디스, LG화학, 장기, 신용등급, A3서, Baa1로]",0,0
7,[코스닥 공시] 국순당 / 녹십자엠에스,"[코스닥, 공시, 국순당, 녹십자엠에스]",0,0
8,[유가증권시장 공시] 선도전기 / 잇츠한불 / 한온시스템 등,"[유가증권시장, 공시, 선도전기, 잇츠한불, 한온시스템, 등]",0,0
9,"은행 방문객 40% '뚝'…""펜도 손세정제로 닦으며 서명""","[은행, 방문객, 40, 뚝, 펜도, 손세정제로, 닦으며, 서명]",0,0


In [43]:
positive_score, negative_score = [], []
p_dict = ['↑', '급상승', '상승', '올랐다']
n_dict = ['↓', '상장폐지', '뚝', '관리종목', '주의보', '낚시', '우려', '둔화', '공포', '사임']

for idx, row in evaluation.iterrows():
    p, n = 0, 0
    for pos in p_dict:
        if pos in row.headline:
            p += 1
            print(pos)
    for neg in n_dict:
        if neg in row.headline:
            n += 1
            print(neg)
    positive_score.append(p)
    negative_score.append(n)

사임
급상승
상승
↓
↓
뚝
상장폐지
관리종목
주의보
올랐다
↑
낚시
공포
급상승
상승
우려
둔화


In [44]:
positive_score

[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0]

In [45]:
negative_score

[0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 3, 0, 0, 1, 1, 0, 2, 0]